# Reverse Sources

Created: 2019.11.11  
Notebook sequence: 12

Finding the predictions of lean from that one place that did it based on facebook follower demographics

---

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd

In [3]:
import util

labels_df = util.nela_load_labels()
mbm_data = util.nela_load_media_bias_monitor()

# Counts

In [4]:
count = 0
bias = [0, 0, 0, 0, 0]
for src in mbm_data:
    value = mbm_data[src]
    if value: 
        count += 1
        if value < -1.2: 
            bias[0] += 1
        elif value < -.4:
            bias[1] += 1
        elif value < .4:
            bias[2] += 1
        elif value < 1.2:
            bias[3] += 1
        else:
            bias[4] += 1
print(count)
print(bias)

82
[5, 35, 23, 4, 15]


In [5]:
mbfc_lbl = "Media Bias / Fact Check, label"
mbfc_bias = [           
    len(list(labels_df[(labels_df[mbfc_lbl] == "left_bias")].Source)),
    len(list(labels_df[(labels_df[mbfc_lbl] == "left_center_bias")].Source)),
    len(list(labels_df[(labels_df[mbfc_lbl] == "least_biased")].Source)),
    len(list(labels_df[(labels_df[mbfc_lbl] == "right_center_bias")].Source)),
    len(list(labels_df[(labels_df[mbfc_lbl] == "right_bias")].Source)),
]
print(mbfc_bias)

[30, 36, 3, 10, 16]


In [6]:
as_lbl = "Allsides, bias_rating"
as_bias = [
    len(list(labels_df[(labels_df[as_lbl] == "Left")].Source)),
    len(list(labels_df[(labels_df[as_lbl] == "Lean Left")].Source)),
    len(list(labels_df[(labels_df[as_lbl] == "Center")].Source)),
    len(list(labels_df[(labels_df[as_lbl] == "Right")].Source)),
    len(list(labels_df[(labels_df[as_lbl] == "Lean Right")].Source)),
]
print(as_bias)

[17, 13, 11, 14, 10]


# Lists

In [7]:
import copy

In [8]:
mbm_sources = {
    "left": [],
    "left-center": [],
    "center": [],
    "right-center": [],
    "right": []
}
alt_mbm_sources = {
    "left": [],
    "left-center": [],
    "center": [],
    "right-center": [],
    "right": []
}

for src in mbm_data:
    value = mbm_data[src]
    if value: 
        if value < -1.2: 
            mbm_sources["left"].append(src)
        elif value < -.4:
            mbm_sources["left-center"].append(src)
        elif value < .4:
            mbm_sources["center"].append(src)
        elif value < 1.2:
            mbm_sources["right-center"].append(src)
        else:
            mbm_sources["right"].append(src)
            
        if value < -.67: 
            alt_mbm_sources["left"].append(src)
        elif value < .67:
            alt_mbm_sources["center"].append(src)
        else:
            alt_mbm_sources["right"].append(src)

mbfc_sources = {
    "left": list(labels_df[(labels_df[mbfc_lbl] == "left_bias")].Source),
    "left-center": list(labels_df[(labels_df[mbfc_lbl] == "left_center_bias")].Source),
    "center": list(labels_df[(labels_df[mbfc_lbl] == "least_biased")].Source),
    "right-center": list(labels_df[(labels_df[mbfc_lbl] == "right_center_bias")].Source),
    "right": list(labels_df[(labels_df[mbfc_lbl] == "right_bias")].Source)
}
alt_mbfc_sources = {
    "left": [],
    "left-center": [],
    "center": [],
    "right-center": [],
    "right": []
}
alt_mbfc_sources["left"] = copy.deepcopy(mbfc_sources["left"])
alt_mbfc_sources["left"].extend(copy.deepcopy(mbfc_sources["left-center"]))
alt_mbfc_sources["center"] = copy.deepcopy(mbfc_sources["center"])
alt_mbfc_sources["right"] = copy.deepcopy(mbfc_sources["right"])
alt_mbfc_sources["right"].extend(copy.deepcopy(mbfc_sources["right-center"]))

as_sources = {
    "left": list(labels_df[(labels_df[as_lbl] == "Left")].Source),
    "left-center": list(labels_df[(labels_df[as_lbl] == "Lean Left")].Source),
    "center": list(labels_df[(labels_df[as_lbl] == "Center")].Source),
    "right-center": list(labels_df[(labels_df[as_lbl] == "Lean Right")].Source),
    "right": list(labels_df[(labels_df[as_lbl] == "Right")].Source)
}
alt_as_sources = {
    "left": [],
    "left-center": [],
    "center": [],
    "right-center": [],
    "right": []
}
alt_as_sources["left"] = copy.deepcopy(as_sources["left"])
alt_as_sources["left"].extend(copy.deepcopy(as_sources["left-center"]))
alt_as_sources["center"] = copy.deepcopy(as_sources["center"])
alt_as_sources["right"] = copy.deepcopy(as_sources["right"])
alt_as_sources["right"].extend(copy.deepcopy(as_sources["right-center"]))

In [9]:
unique = {
    "left": [],
    "left-center": [],
    "center": [],
    "right-center": [],
    "right": []
}

votes = {
    "left": {},
    "left-center": {},
    "center": {},
    "right-center": {},
    "right": {}
}

def tally_votes(key, sources):
    global votes
    
    for src in sources[key]:
        if src not in votes[key]:
            votes[key][src] = 1
        else:
            votes[key][src] += 1
            
mbm_use = alt_mbm_sources
mbfc_use = alt_mbfc_sources
as_use = alt_as_sources

for key in ["left", "left-center", "center", "right-center", "right"]:
    print("-----",key,"-----")
    print(mbm_use[key])
    print(mbfc_use[key])
    print(as_use[key])
    print("\n\n")
    
    combined = []
    combined.extend(mbm_use[key])
    combined.extend(mbfc_use[key])
    combined.extend(as_use[key])
    combined = list(set(combined))
    
    unique[key] = combined
    
    tally_votes(key, mbm_use)
    tally_votes(key, mbfc_use)
    tally_votes(key, as_use)

----- left -----
['Addicting Info', 'Al Jazeera', 'Alternet', 'Daily Beast', 'Daily Kos', 'Faking News', 'Feministing Blog', 'FiveThirtyEight', 'France24', 'MSNBC', 'Mercury News', 'MotherJones', 'NPR', 'New Yorker', 'Pink News UK', 'Politico', 'Raw story', 'Slate', 'Spiegel', 'Sputnik', 'Talking Points Memo', 'The Atlantic', 'ThinkProgress', 'Vox']
['Addicting Info', 'Alternet', 'CNN', 'Crooks and Liars', 'Daily Beast', 'Daily Kos', 'Feministing Blog', 'Forward Progessives', 'MSNBC', 'Media Matters for America', 'MotherJones', 'New Yorker', 'Newsweek', 'Palmer Report', 'Pink News UK', 'Pravada Report', 'Raw Story', 'RightWingWatch', 'Salon', 'Shadow Proof', 'Shareblue', 'Slate', 'Talking Points Memo', 'Telesur TV', 'The Daily Mirror', 'The Daily Record', 'The Huffington Post', 'ThinkProgress', 'Vox', 'Washington Monthly', 'ABC News', 'Al Jazeera', 'BBC', 'Birmingham Mail', 'Business Insider', 'Buzzfeed', 'CBS News', 'CNBC', 'Chicago Sun-Times', 'Democracy 21', 'FiveThirtyEight', 'Fran

In [10]:
for key in votes.keys():
    print("-----",key,"-----")
    for src in votes[key]:
        print(src,votes[key][src])
        
    print("\n\n")

----- left -----
Addicting Info 2
Al Jazeera 2
Alternet 3
Daily Beast 3
Daily Kos 3
Faking News 1
Feministing Blog 2
FiveThirtyEight 2
France24 2
MSNBC 3
Mercury News 2
MotherJones 3
NPR 2
New Yorker 3
Pink News UK 2
Politico 1
Raw story 1
Slate 3
Spiegel 2
Sputnik 1
Talking Points Memo 2
The Atlantic 3
ThinkProgress 3
Vox 3
CNN 2
Crooks and Liars 1
Forward Progessives 1
Media Matters for America 2
Newsweek 2
Palmer Report 1
Pravada Report 1
Raw Story 2
RightWingWatch 1
Salon 2
Shadow Proof 1
Shareblue 1
Telesur TV 1
The Daily Mirror 1
The Daily Record 1
The Huffington Post 2
Washington Monthly 2
ABC News 2
BBC 1
Birmingham Mail 1
Business Insider 1
Buzzfeed 2
CBS News 2
CNBC 1
Chicago Sun-Times 2
Democracy 21 1
Fusion 1
Interpreter Mag 1
New York Daily News 2
PBS 1
SkyNewsPolitics 1
SkyNewsUS 1
The Denver Post 1
The Guardian 2
The Hill 1
The Independent 1
The Intercept 2
The Irish Times 1
The Moscow Times 1
The New York Times 2
The Verge 2
USA Today 1
Washington Post 2
Wings Over Scot

In [11]:
def find_src(src, sources, name):
    for key in sources.keys():
        if src in sources[key]:
            print("\tFound",src,"for",name,"in",key)

for src in mbm_sources["center"]:
    print(src, mbm_data[src])
    
    find_src(src, mbfc_sources, "mbfc")
    find_src(src, as_sources, "as")

ABC News -0.117
	Found ABC News for mbfc in left-center
	Found ABC News for as in left-center
BBC UK -0.241
Birgmingham Mail 0.156
Business Insider -0.215
	Found Business Insider for mbfc in left-center
	Found Business Insider for as in center
Buzzfeed -0.268
	Found Buzzfeed for mbfc in left-center
	Found Buzzfeed for as in left-center
CBS News 0.244
	Found CBS News for mbfc in left-center
	Found CBS News for as in left-center
CNBC -0.075
	Found CNBC for mbfc in left-center
	Found CNBC for as in center
CNN -0.342
	Found CNN for mbfc in left
	Found CNN for as in left-center
Chicago Sun-Times -0.204
	Found Chicago Sun-Times for mbfc in left-center
	Found Chicago Sun-Times for as in left-center
Common Dreams -0.035
Daily Mail -0.021
	Found Daily Mail for as in right
Foreign Policy -0.174
	Found Foreign Policy for mbfc in center
New York Daily News -0.359
	Found New York Daily News for mbfc in left-center
	Found New York Daily News for as in left
New York Post -0.138
	Found New York Post f

In [12]:
for src in as_sources["center"]:
    print(src)
    
    find_src(src, mbm_sources, "mbm")
    find_src(src, mbfc_sources, "mbfc")

Al Jazeera
	Found Al Jazeera for mbm in left-center
	Found Al Jazeera for mbfc in left-center
BBC
	Found BBC for mbm in left-center
	Found BBC for mbfc in left-center
Business Insider
	Found Business Insider for mbm in center
	Found Business Insider for mbfc in left-center
CNBC
	Found CNBC for mbm in center
	Found CNBC for mbfc in left-center
FiveThirtyEight
	Found FiveThirtyEight for mbm in left-center
	Found FiveThirtyEight for mbfc in left-center
NPR
	Found NPR for mbm in left-center
	Found NPR for mbfc in left-center
PBS
	Found PBS for mbm in left-center
	Found PBS for mbfc in left-center
Real Clear Politics
	Found Real Clear Politics for mbm in center
	Found Real Clear Politics for mbfc in right-center
Reuters
	Found Reuters for mbm in left-center
	Found Reuters for mbfc in center
The Hill
	Found The Hill for mbm in left-center
	Found The Hill for mbfc in left-center
USA Today
	Found USA Today for mbm in center
	Found USA Today for mbfc in left-center


In [13]:
for src in mbfc_sources["center"]:
    print(src)
    
    find_src(src, mbm_sources, "mbm")
    find_src(src, as_sources, "as")

Foreign Policy
	Found Foreign Policy for mbm in center
Politico
	Found Politico for mbm in left-center
Reuters
	Found Reuters for mbm in left-center
	Found Reuters for as in center


## Master List

In [14]:
final_source_list = {
    "left": [],
    "center": [],
    "right": []
}
final_source_list_binary = {
    "biased": [],
    "center": []
}

First collect all sources that have more than two votes

In [15]:
for key in ["left", "center", "right"]:
    for src in votes[key]:
        if votes[key][src] > 1:
            final_source_list[key].append(src)

# print(json.dumps(final_source_list, indent=2))

Take all center sources and add them to center if not extreme by any of the other sources

In [16]:
def sufficiently_agreed_src(src, sources):
    """ return true if not in left or right, and false if so """
    for key in sources.keys():
        if src in sources[key]:
            #print("\tFound",src,"for",name,"in",key)
            if key == "left" or key == "right":
                print("\tStrong disagreement!")
                return False
            else: 
                return True
        return True


def add_sufficient_from_sources(sources, sources_check1, sources_check2):
    for src in sources["center"]:
        
        source1_good = sufficiently_agreed_src(src, sources_check1)
        source2_good = sufficiently_agreed_src(src, sources_check2)
        
        if source1_good and source2_good:
            if src not in final_source_list["center"]:
                final_source_list["center"].append(src)
                if src in final_source_list["left"]:
                    final_source_list["left"].remove(src)
                if src in final_source_list["right"]:
                    final_source_list["right"].remove(src)
        
        
add_sufficient_from_sources(mbm_sources, mbfc_sources, as_sources)
add_sufficient_from_sources(mbfc_sources, mbm_sources, as_sources)
add_sufficient_from_sources(as_sources, mbfc_sources, mbm_sources)

	Strong disagreement!
	Strong disagreement!
	Strong disagreement!
	Strong disagreement!
	Strong disagreement!


In [17]:
print(json.dumps(final_source_list, indent=2))

total = 0
for key in final_source_list:
    print(key, len(final_source_list[key]))
    total += len(final_source_list[key])
print(total)

{
  "left": [
    "Addicting Info",
    "Alternet",
    "Daily Beast",
    "Daily Kos",
    "Feministing Blog",
    "France24",
    "MSNBC",
    "Mercury News",
    "MotherJones",
    "New Yorker",
    "Pink News UK",
    "Slate",
    "Spiegel",
    "Talking Points Memo",
    "The Atlantic",
    "ThinkProgress",
    "Vox",
    "CNN",
    "Media Matters for America",
    "Newsweek",
    "Raw Story",
    "Salon",
    "The Huffington Post",
    "Washington Monthly",
    "New York Daily News",
    "The Guardian",
    "The Intercept",
    "The New York Times",
    "The Verge",
    "Washington Post",
    "Yahoo News"
  ],
  "center": [
    "BBC",
    "Business Insider",
    "CNBC",
    "Foreign Policy",
    "PBS",
    "Real Clear Politics",
    "Reuters",
    "The Hill",
    "USA Today",
    "ABC News",
    "BBC UK",
    "Birgmingham Mail",
    "Buzzfeed",
    "CBS News",
    "Chicago Sun-Times",
    "Common Dreams",
    "Daily Mail",
    "New York Post",
    "The Daily Express",
    "The De

# Split the sources up into folds

In [18]:

counts = {}

for key in final_source_list:
    print("-----",key,"-----")
    for src in final_source_list[key]:
        count = util.nela_count_articles_from_source(src).iloc[0,0]
        counts[src] = count
        if count < 300:
            print(src, "REMOVED")
            final_source_list[key].remove(src)
        else:
            print(src, count)
        
        

----- left -----
Addicting Info 429
Alternet 4816
Daily Beast 6634
Daily Kos 994
Feministing Blog REMOVED
MSNBC 6604
Mercury News 4828
MotherJones 1128
New Yorker REMOVED
Slate 514
Spiegel 4171
Talking Points Memo 5846
The Atlantic 1757
ThinkProgress 4819
Vox 4288
CNN 8202
Media Matters for America 2316
Newsweek 9411
Raw Story 3719
Salon 1702
The Huffington Post 5586
Washington Monthly 551
New York Daily News 2042
The Guardian 2195
The Intercept 1268
The New York Times 5471
The Verge 5951
Washington Post 1252
Yahoo News 1666
----- center -----
BBC 16416
Business Insider 445
CNBC 2426
Foreign Policy 702
PBS 1113
Real Clear Politics 7247
Reuters 3929
The Hill 1968
USA Today 5968
ABC News 2808
BBC UK 5504
Birgmingham Mail REMOVED
CBS News 5397
Chicago Sun-Times 2113
Common Dreams REMOVED
New York Post 25407
The Daily Express 1585
The Denver Post 4503
The Irish Times 3827
The Moscow Times 1137
The Telegraph 33763
Politico 629
Al Jazeera 4522
FiveThirtyEight 556
NPR 5515
----- right -----
B

In [19]:
total = 0
for key in final_source_list:
    print(key, len(final_source_list[key]))
    total += len(final_source_list[key])
print(total)

left 29
center 25
right 16
70


In [20]:
left = len(final_source_list["left"])
center = len(final_source_list["center"])
right = len(final_source_list["right"])

In [21]:
num_folds = 10

print(left / num_folds)
print(center / num_folds)
print(right / num_folds)

2.9
2.5
1.6


# BELOW STEP IS WHERE DIFFERENT SELECTION BASES ARE DETERMINED

In [60]:
from sklearn.model_selection import KFold

def generate_fold_sources(sources_list, random_state=13):
    folds = []
    
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
    for train_index, test_index in kf.split(sources_list):
        #left_folds.append(test_index)
        sources = []
        for index in test_index:
            sources.append(sources_list[index])

        folds.append(sources)
        
    return folds

# =================================================================
# Uncomment to remove spiegal
final_source_list["left"].remove("Spiegel")



# =================================================================





# =================================================================



left_folds = generate_fold_sources(final_source_list["left"])
center_folds = generate_fold_sources(final_source_list["center"])
right_folds = generate_fold_sources(final_source_list["right"])

final_folds = []
for i in range(0, 10):
    all_fold_sources = {
        "left": left_folds[i],
        "center": center_folds[i],
        "right": right_folds[i]
    }
    final_folds.append(all_fold_sources)
    

# kf = KFold(n_splits=10, shuffle=True, random_state=13)
# for train_index, test_index in kf.split(final_source_list["left"]):
#     #left_folds.append(test_index)
#     sources = []
#     for index in test_index:
#         sources.append(final_source_list["left"][index])
    
#     left_folds.append(sources)
        
    
print(left_folds)
print(center_folds)
print(right_folds)

print("")
print(final_folds)

[['Media Matters for America', 'Raw Story', 'New York Daily News'], ['Alternet', 'MotherJones', 'The Intercept'], ['Pink News UK', 'The Huffington Post', 'The New York Times'], ['Addicting Info', 'France24', 'Yahoo News'], ['Slate', 'The Atlantic', 'The Verge'], ['MSNBC', 'Vox', 'CNN'], ['Daily Kos', 'ThinkProgress', 'The Guardian'], ['Daily Beast', 'Mercury News', 'Washington Monthly'], ['Talking Points Memo', 'Washington Post'], ['Newsweek', 'Salon']]
[['Business Insider', 'USA Today', 'NPR'], ['The Hill', 'Buzzfeed', 'New York Post'], ['BBC', 'Chicago Sun-Times', 'The Denver Post'], ['PBS', 'ABC News', 'Politico'], ['Real Clear Politics', 'The Telegraph', 'Al Jazeera'], ['Daily Mail', 'The Moscow Times'], ['Foreign Policy', 'CBS News'], ['CNBC', 'Reuters'], ['BBC UK', 'FiveThirtyEight'], ['The Daily Express', 'The Irish Times']]
[['The American Conservative', 'The Washington Examiner'], ['Daily Signal', 'TheBlaze'], ['Infowars', 'RedState'], ['The Daily Caller', 'Investors Business 

In [61]:
def save_selection(tag=""):
    with open(f"../data/cache/{tag}folds_selection.json", 'w') as outfile:
        json.dump(final_folds, outfile)

#save_selection()
save_selection("nospiegel")

In [23]:
#with open("../data/cache/folds_selection.json", 'w') as outfile:
#    json.dump(final_folds, outfile)

In [24]:
def find_and_pretty_class(sources, source):
    if source in sources["left"]:
        return "Left"
    elif source in sources["left-center"]:
        return "Left-center"
    elif source in sources["center"]:
        return "Center"
    elif source in sources["right-center"]:
        return "Right-center"
    elif source in sources["right"]:
        return "Right"
    else:
        return "--"

def find_in_mbm(source):
    if source in mbm_data and mbm_data[source] != False:
        return mbm_data[source]
    else:
        return "--"

In [25]:
mbm_data["CNN"]

-0.342

In [26]:
fold_table_data = []
folds_left = []
folds_center = []
folds_right = []

for index, fold in enumerate(final_folds):
    for source in fold["left"]:
        folds_left.append(
            {
                "Source":source, 
                "Classification":"Left", 
                "AllSides": find_and_pretty_class(as_sources, source),
                "Media Bias/Fact Check": find_and_pretty_class(mbfc_sources, source),
                "Media Bias Monitor": str(find_in_mbm(source)),
                "Fold": index
            })
    for source in fold["center"]:
        folds_center.append(
            {
                "Source":source, 
                "Classification":"Center", 
                "AllSides": find_and_pretty_class(as_sources, source),
                "Media Bias/Fact Check": find_and_pretty_class(mbfc_sources, source),
                "Media Bias Monitor": str(find_in_mbm(source)),
                "Fold": index
            })
    for source in fold["right"]:
        folds_right.append(
            {
                "Source":source, 
                "Classification":"Right", 
                "AllSides": find_and_pretty_class(as_sources, source),
                "Media Bias/Fact Check": find_and_pretty_class(mbfc_sources, source),
                "Media Bias Monitor": str(find_in_mbm(source)),
                "Fold": index
            })

fold_table_data.extend(folds_left)
fold_table_data.extend(folds_center)
fold_table_data.extend(folds_right)

fold_df = pd.DataFrame(fold_table_data)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(fold_df)

,Source,Classification,AllSides,Media Bias/Fact Check,Media Bias Monitor,Fold
0,CNN,Left,Left-center,Left,-0.342,0
1,Washington Monthly,Left,Left-center,Left,--,0
2,The New York Times,Left,Left-center,Left-center,-0.569,0
3,Alternet,Left,Left,Left,-1.175,1
4,Pink News UK,Left,--,Left,-0.973,1
5,The Intercept,Left,Left,Left-center,--,1
6,MotherJones,Left,Left,Left,-1.326,2
7,Newsweek,Left,Left-center,Left,-0.468,2
8,Yahoo News,Left,Left,Left-center,0.052,2
9,Addicting Info,Left,--,Left,-0.822,3


In [27]:
fold_df.to_csv("fold_source_table.csv", index=False)

In [28]:
def convert_to_num(label):
    if label == "Right":
        return 2
    elif label == "Right-center":
        return 1
    elif label == "Center":
        return 0
    elif label == "Left-center":
        return -1
    else: 
        return -2

def convert_mbm(value):
    if value is None:
        return None
    
    if value < -1.2: 
        return -2
    elif value < -.4:
        return -1
    elif value < .4:
        return 0
    elif value < 1.2:
        return 1
    else:
        return 2
    
def mbm_class(value):
    if value == -2:
        return "Left"
    elif value == -1:
        return "Left-center"
    elif value == 0:
        return "Center"
    elif value == 1:
        return "Right-center"
    elif value == 2:
        return "Right"
    else:
        return "--"

disagreement = []
mbm_classifications = []

for index, row in fold_df.iterrows():
    classification = row['Classification']
    as_r = row['AllSides']
    mbfc_r = row['Media Bias/Fact Check']
    
    mbm_r = row['Media Bias Monitor']

    if mbm_r == "--":
        mbm_r = None
    else:
        mbm_r = float(row['Media Bias Monitor'])

    difference = 0
    
    class_val = convert_to_num(classification)
    as_r_val = convert_to_num(as_r)
    mbfc_r_val = convert_to_num(mbfc_r)
    mbm_r_val = convert_mbm(mbm_r)
    mbm_classification = mbm_class(mbm_r_val)
    mbm_classifications.append(mbm_classification)
    
    print(classification, as_r, mbfc_r, mbm_r)
    print(class_val, as_r_val, mbfc_r_val, mbm_r_val)
    
    if as_r != "--":
        difference += abs(class_val - as_r_val)
    if mbfc_r != "--":
        difference += abs(class_val - mbfc_r_val)
    if mbm_r is not None:
        difference += abs(class_val - mbm_r_val)
    print(difference)
    
    disagreement.append(difference)
    

Left Left-center Left -0.342
-2 -1 -2 0
3
Left Left-center Left None
-2 -1 -2 None
1
Left Left-center Left-center -0.569
-2 -1 -1 -1
3
Left Left Left -1.175
-2 -2 -2 -1
1
Left -- Left -0.973
-2 -2 -2 -1
1
Left Left Left-center None
-2 -2 -1 None
1
Left Left Left -1.326
-2 -2 -2 -2
0
Left Left-center Left -0.468
-2 -1 -2 -1
2
Left Left Left-center 0.052
-2 -2 -1 0
3
Left -- Left -0.822
-2 -2 -2 -1
1
Left Left Left -0.172
-2 -2 -2 0
2
Left Left Left-center -0.359
-2 -2 -1 0
3
Left -- Left-center -0.869
-2 -2 -1 -1
2
Left Left Left -0.878
-2 -2 -2 -1
1
Left -- Left -1.178
-2 -2 -2 -1
1
Left Left Left -1.35
-2 -2 -2 -2
0
Left Left Left -1.218
-2 -2 -2 -2
0
Left Left-center Left-center -0.467
-2 -1 -1 -1
3
Left Left Left -1.217
-2 -2 -2 -2
0
Left Left Left -1.063
-2 -2 -2 -1
1
Left Left-center Left-center -0.444
-2 -1 -1 -1
3
Left Left Left -0.827
-2 -2 -2 -1
1
Left Left-center Left-center -0.881
-2 -1 -1 -1
3
Left Left Left -0.476
-2 -2 -2 -1
1
Left -- Left-center -0.748
-2 -2 -1 -1
2
Left

In [55]:
fold_df_disagreement = fold_df.copy()
fold_df_disagreement["Disagreement"] = disagreement
fold_df_disagreement["Media Bias Monitor Class"] = mbm_classifications
fold_df_disagreement.columns
fold_df_disagreement = fold_df_disagreement[['Source', 'Classification', 'AllSides', 'Media Bias/Fact Check',
       'Media Bias Monitor Class', 'Media Bias Monitor', 'Fold', 'Disagreement']]

In [56]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(fold_df_disagreement)

,Source,Classification,AllSides,Media Bias/Fact Check,Media Bias Monitor Class,Media Bias Monitor,Fold,Disagreement
0,CNN,Left,Left-center,Left,Center,-0.342,0,3
1,Washington Monthly,Left,Left-center,Left,--,--,0,1
2,The New York Times,Left,Left-center,Left-center,Left-center,-0.569,0,3
3,Alternet,Left,Left,Left,Left-center,-1.175,1,1
4,Pink News UK,Left,--,Left,Left-center,-0.973,1,1
5,The Intercept,Left,Left,Left-center,--,--,1,1
6,MotherJones,Left,Left,Left,Left,-1.326,2,0
7,Newsweek,Left,Left-center,Left,Left-center,-0.468,2,2
8,Yahoo News,Left,Left,Left-center,Center,0.052,2,3
9,Addicting Info,Left,--,Left,Left-center,-0.822,3,1


In [57]:
fold_df_disagreement.to_csv("../data/cache/bias_folds_detail.csv")

In [59]:
dict(fold_df_disagreement.iloc[0])

{'Source': 'CNN',
 'Classification': 'Left',
 'AllSides': 'Left-center',
 'Media Bias/Fact Check': 'Left',
 'Media Bias Monitor Class': 'Center',
 'Media Bias Monitor': '-0.342',
 'Fold': 0,
 'Disagreement': 3}

# Reliability

In [31]:
labels_df = util.nela_load_labels()

def get_reliable_for_source(source):
    if source == "os":
        reliable_lbl = "Open Sources, reliable"
        unreliable_lbl = "Open Sources, unreliable"

        unreliable = list(labels_df[labels_df[unreliable_lbl].notnull()].Source)
        reliable = list(labels_df[labels_df[reliable_lbl].notnull()].Source)

    if source == "mbfc":
        lbl = "Media Bias / Fact Check, factual_reporting"

        reliable = list(labels_df[labels_df[lbl] > 3].Source)
        unreliable = list(labels_df[labels_df[lbl] <= 3].Source)

    if source == "ng":
        ng_lbl = "NewsGuard, overall_class"

        reliable = list(labels_df[labels_df[ng_lbl] == 1.0].Source)
        unreliable = list(labels_df[labels_df[ng_lbl] == 0.0].Source)
    return reliable, unreliable

os_r, os_u = get_reliable_for_source("os")
mbfc_r, mbfc_u = get_reliable_for_source("mbfc")
ng_r, ng_u = get_reliable_for_source("ng")

In [32]:
os_r

['Alternet', 'Fusion', 'oann']

In [33]:
mbfc_r

['ABC News',
 'Al Jazeera',
 'BBC',
 'Business Insider',
 'CBS News',
 'CNBC',
 'Chicago Sun-Times',
 'Daily Beast',
 'Democracy 21',
 'Evening Standard',
 'FiveThirtyEight',
 'Foreign Policy',
 'Fortune',
 'Forward Progessives',
 'France24',
 'Hot Air',
 'Interpreter Mag',
 'Media Matters for America',
 'Mercury News',
 'MotherJones',
 'NPR',
 'New Yorker',
 'PBS',
 'Pink News UK',
 'Politico',
 'Real Clear Politics',
 'Reuters',
 'Salon',
 'Shadow Proof',
 'SkyNewsPolitics',
 'SkyNewsUS',
 'Slate',
 'Spiegel',
 'Talking Points Memo',
 'Tass',
 'The American Conservative',
 'The Atlantic',
 'The Denver Post',
 'The Fiscal Times',
 'The Guardian',
 'The Hill',
 'The Huffington Post',
 'The Independent',
 'The Intercept',
 'The Irish Times',
 'The Moscow Times',
 'The New York Times',
 'The Telegraph',
 'The Verge',
 'The Washington Examiner',
 'USA Today',
 'Vox',
 'Washington Monthly',
 'Washington Post',
 'Wings Over Scotland',
 'iPolitics']

In [34]:
ng_r

['ABC News',
 'Alternet',
 'BBC',
 'Bearing Arms',
 'Business Insider',
 'Buzzfeed',
 'CBS News',
 'CNBC',
 'CNN',
 'CNS News',
 'Chicago Sun-Times',
 'Crooks and Liars',
 'Daily Beast',
 'Daily Signal',
 'FT Westminster Blog',
 'FiveThirtyEight',
 'Foreign Policy',
 'Fortune',
 'Fox News',
 'Investors Business Daily',
 'MSNBC',
 'Media Matters for America',
 'Mercury News',
 'MotherJones',
 'NPR',
 'National Review',
 'New York Daily News',
 'New York Post',
 'New Yorker',
 'News Busters',
 'Newsweek',
 'Observer',
 'PBS',
 'Politico',
 'Politicus USA',
 'Raw Story',
 'Real Clear Politics',
 'Reuters',
 'Salon',
 'Slate',
 'Talking Points Memo',
 'The American Conservative',
 'The Atlantic',
 'The Daily Caller',
 'The Denver Post',
 'The Guardian',
 'The Hill',
 'The Huffington Post',
 'The Independent',
 'The Intercept',
 'The New York Times',
 'The Verge',
 'The Washington Examiner',
 'ThinkProgress',
 'USA Today',
 'Vox',
 'WSJ Washington Wire',
 'Washington Monthly',
 'Washington 

In [35]:
total_claimed_r = []
total_claimed_r.extend(os_r)
total_claimed_r.extend(mbfc_r)
total_claimed_r.extend(ng_r)
total_claimed_r = list(set(total_claimed_r))
len(total_claimed_r)

81

In [36]:
total_claimed_u = []
total_claimed_u.extend(os_u)
total_claimed_u.extend(mbfc_u)
total_claimed_u.extend(ng_u)
total_claimed_u = list(set(total_claimed_u))
len(total_claimed_u)

72

In [37]:
def count_in(source, reliable=True):
    count = 0
    if reliable:
        if source in os_r:
            count += 1
        if source in mbfc_r:
            count += 1
        if source in ng_r:
            count += 1
    else:
        if source in os_u:
            count += 1
        if source in mbfc_u:
            count += 1
        if source in ng_u:
            count += 1
    return count

reliable_counts = {}
unreliable_counts = {}

for source in total_claimed_r:
    reliable_counts[source] = count_in(source, True)

for source in total_claimed_u:
    unreliable_counts[source] = count_in(source, False)

In [38]:
actual_reliable = []
for source in reliable_counts:
    print(source, reliable_counts[source])
    if source in unreliable_counts.keys():
        print("PROBLEM WITH", source)
        if reliable_counts[source] == 1:
            print("not adding")
            continue
    print("(added)")
    actual_reliable.append(source)

Observer 1
(added)
Raw Story 1
PROBLEM WITH Raw Story
not adding
Forward Progessives 1
(added)
Business Insider 2
(added)
BBC 2
(added)
Vox 2
(added)
The Intercept 2
(added)
Media Matters for America 2
(added)
New Yorker 2
(added)
The Hill 2
(added)
Fusion 1
PROBLEM WITH Fusion
not adding
USA Today 2
(added)
National Review 1
PROBLEM WITH National Review
not adding
The Independent 2
(added)
The Verge 2
(added)
Slate 2
(added)
CNN 1
PROBLEM WITH CNN
not adding
Salon 2
(added)
The Telegraph 1
(added)
Mercury News 2
(added)
Talking Points Memo 2
(added)
Evening Standard 1
(added)
Pink News UK 1
(added)
The Fiscal Times 1
(added)
Shadow Proof 1
(added)
Al Jazeera 1
PROBLEM WITH Al Jazeera
not adding
Real Clear Politics 2
(added)
Newsweek 1
PROBLEM WITH Newsweek
not adding
The Washington Examiner 2
PROBLEM WITH The Washington Examiner
(added)
Buzzfeed 1
PROBLEM WITH Buzzfeed
not adding
Wings Over Scotland 1
(added)
Daily Signal 1
PROBLEM WITH Daily Signal
not adding
Reuters 2
(added)
WSJ Wa

In [39]:
actual_unreliable = []
for source in unreliable_counts:
    print(source, unreliable_counts[source])
    if source in reliable_counts.keys():
        print("PROBLEM WITH", source)
        if unreliable_counts[source] == 1:
            print("not adding")
            continue
    print("(added)")
    actual_unreliable.append(source)

RT 1
(added)
The Sun 1
(added)
True Activist 1
(added)
21stCenturyWire 1
(added)
Raw Story 1
PROBLEM WITH Raw Story
not adding
ThinkProgress 1
PROBLEM WITH ThinkProgress
not adding
The Political Insider 3
(added)
The Gateway Pundit 2
(added)
Pravada Report 1
(added)
Politicus USA 1
PROBLEM WITH Politicus USA
not adding
Crooks and Liars 1
PROBLEM WITH Crooks and Liars
not adding
Sputnik 2
(added)
The Daily Caller 1
PROBLEM WITH The Daily Caller
not adding
Bearing Arms 1
PROBLEM WITH Bearing Arms
not adding
Shareblue 3
(added)
Al Jazeera 1
PROBLEM WITH Al Jazeera
not adding
FrontPage Magazine 1
(added)
Instapundit 1
(added)
Fox News 1
PROBLEM WITH Fox News
not adding
Counter Current News 1
(added)
Live Action 2
(added)
Newsweek 1
PROBLEM WITH Newsweek
not adding
Birmingham Mail 1
(added)
New York Daily News 1
PROBLEM WITH New York Daily News
not adding
The Daily Mirror 1
(added)
The Washington Examiner 1
PROBLEM WITH The Washington Examiner
not adding
The Daily Star 1
(added)
Feministing

In [40]:
print(len(actual_reliable))
print(len(actual_unreliable))

61
50


In [41]:
actual_reliable

['Observer',
 'Forward Progessives',
 'Business Insider',
 'BBC',
 'Vox',
 'The Intercept',
 'Media Matters for America',
 'New Yorker',
 'The Hill',
 'USA Today',
 'The Independent',
 'The Verge',
 'Slate',
 'Salon',
 'The Telegraph',
 'Mercury News',
 'Talking Points Memo',
 'Evening Standard',
 'Pink News UK',
 'The Fiscal Times',
 'Shadow Proof',
 'Real Clear Politics',
 'The Washington Examiner',
 'Wings Over Scotland',
 'Reuters',
 'WSJ Washington Wire',
 'The Huffington Post',
 'The Guardian',
 'CNBC',
 'FT Westminster Blog',
 'CBS News',
 'France24',
 'FiveThirtyEight',
 'News Busters',
 'PBS',
 'Interpreter Mag',
 'MotherJones',
 'The American Conservative',
 'Daily Beast',
 'Hot Air',
 'Alternet',
 'The Denver Post',
 'Washington Monthly',
 'NPR',
 'Chicago Sun-Times',
 'Democracy 21',
 'iPolitics',
 'oann',
 'The Moscow Times',
 'SkyNewsPolitics',
 'The Irish Times',
 'The New York Times',
 'Fortune',
 'SkyNewsUS',
 'Foreign Policy',
 'Politico',
 'The Atlantic',
 'Tass',
 '

In [42]:
actual_unreliable

['RT',
 'The Sun',
 'True Activist',
 '21stCenturyWire',
 'The Political Insider',
 'The Gateway Pundit',
 'Pravada Report',
 'Sputnik',
 'Shareblue',
 'FrontPage Magazine',
 'Instapundit',
 'Counter Current News',
 'Live Action',
 'Birmingham Mail',
 'The Daily Mirror',
 'The Daily Star',
 'Feministing Blog',
 'True Pundit',
 'TheAntiMedia',
 'GlobalResearch',
 'The Daily Express',
 'Palmer Report',
 'The Conservative Tree House',
 'Waking Times',
 'Addicting Info',
 'The Right Scoop',
 'TheBlaze',
 'Pamela Geller Report',
 'The Daily Record',
 'Veterans Today',
 'Natural News',
 'LewRockwell',
 'Daily Kos',
 'RightWingWatch',
 'Russia-Insider',
 'Breitbart',
 'Prison Planet',
 'Bipartisan Report',
 'Daily Mail',
 'Activist Post',
 'Infowars',
 'HumansAreFree',
 'Western Journal',
 'sott.net',
 'Intellihub',
 'Telesur TV',
 'Drudge Report',
 'RedState',
 'NODISINFO',
 'The Duran']

In [43]:
num_folds = 10

print(len(actual_reliable) / num_folds)
print(len(actual_unreliable) / num_folds)

6.1
5.0


In [44]:
from sklearn.model_selection import KFold

def generate_reliability_fold_sources(sources_list, random_state=13):
    folds = []
    
    kf = KFold(n_splits=10, shuffle=True, random_state=random_state)
    for train_index, test_index in kf.split(sources_list):
        #left_folds.append(test_index)
        sources = []
        for index in test_index:
            sources.append(sources_list[index])

        folds.append(sources)
        
    return folds


reliable_f = generate_fold_sources(actual_reliable)
unreliable_f = generate_fold_sources(actual_unreliable)

final_reliability_folds = []
for i in range(0, 10):
    all_fold_sources = {
        "reliable": reliable_f[i],
        "unreliable": unreliable_f[i]
    }
    final_reliability_folds.append(all_fold_sources)

In [45]:
print(reliable_f)
print(unreliable_f)
print("")
print(final_reliability_folds)

[['Observer', 'Vox', 'The Telegraph', 'SkyNewsUS', 'Foreign Policy', 'Politico', 'Tass'], ['Media Matters for America', 'Wings Over Scotland', 'FT Westminster Blog', 'CBS News', 'France24', 'Democracy 21'], ['Mercury News', 'Evening Standard', 'The Guardian', 'News Busters', 'The Denver Post', 'ABC News'], ['BBC', 'The Intercept', 'New Yorker', 'Shadow Proof', 'Hot Air', 'The Atlantic'], ['The Hill', 'Slate', 'MotherJones', 'Washington Monthly', 'The Irish Times', 'The New York Times'], ['Forward Progessives', 'The Washington Examiner', 'Reuters', 'CNBC', 'oann', 'Washington Post'], ['The Fiscal Times', 'Real Clear Politics', 'FiveThirtyEight', 'Alternet', 'NPR', 'SkyNewsPolitics'], ['USA Today', 'The Verge', 'Salon', 'The American Conservative', 'iPolitics', 'Spiegel'], ['Business Insider', 'WSJ Washington Wire', 'The Huffington Post', 'Interpreter Mag', 'Chicago Sun-Times', 'Fortune'], ['The Independent', 'Talking Points Memo', 'Pink News UK', 'PBS', 'Daily Beast', 'The Moscow Times'

In [46]:
with open("../data/cache/reliability_folds_selection.json", 'w') as outfile:
    json.dump(final_reliability_folds, outfile)

In [47]:
"Hot Air" in os_r

False

In [48]:
"Hot Air" in ng_r

False

In [49]:
"Hot Air" in mbfc_r

True

In [50]:
"Hot Air" in os_u

False

In [51]:
"Hot Air" in ng_u

False